# Scattering and particle creation in 1D Ising Field Theory. 

This challenge is based on https://arxiv.org/pdf/2505.03111.

The goal of this challenge is to study the time evolution of two localized single-particle states within a 1D Ising field theory as they evolve towards and scatter off each other. For our choice of coupling parameters and initial energy, an inelastic process can occur in which energy is converted to mass -- specifically, the scattered state contains a particle of higher mass than the incoming state. A goal of this challenge is to detect a signal of the aforementioned inelastic scattering. 

The scattering processes are visualized in the picture below: two "light" particles move towards each other, collide, and either: 1) scatter elastically, 2) scatter *in*elastically (with an out-going heavier, slower-moving particle moving to the  **left**), or 3) scatter  *in*elastically (with an out-going heavier, slower-moving particle moving to the  **right**). All processes appear in the figure because they occur in superposition.  


<div>
<img src="figs/particle_creation.png" width="400"/>
</div>

The Hamiltonian for this field theory on a lattice with `L` sites is given by:

$$H_{\text{ising}} = -\sum_{n=0}^{L-2} Z_{n} Z_{n+1} - \sum_{n=0}^{L-1}\big( g_{x} X_n + g_{z} Z_n \big).$$

We will fix $g_x = 1.25$ and $g_z = 0.15$. The resulting system features two stable particles: one "light" particle $|1\rangle$ with mass $m$, and one "heavy" particle $|2\rangle$ with mass $M>m$.

<div class="alert alert-block alert-success">

### **Your Task:** 
### Implement a quantum simulation of high-energy particle collision in 1D Ising field theory.

In this notebook, we will first run matrix product state (MPS) simulations of scattering on a small lattice (PART I), and then we will run hardware experiments of scattering on a large lattice (PART II). You will have freedom to improve upon the experiment with error mitigation and post-processing techniques. 

### For both MPS simulation and hardware experiments, there are a few basic steps involved: 
1. **Prepare an initial state that has two wavepackets with definite momentum $k_0$.** We will call each wavepacket state $|W(k_0)\rangle$ due to the fact that it is a special case of the $W$-state, and we will prepare two of them separated over space within the lattice and with opposite momentum, such that they are moving towards each other. 
2. **Minimize the energy of the initial state without changing the momentum.** This step ensures that we begin with the lowest-energy excited state that corresponds to the single-particle state with largest mass in our Ising theory, set by parameters $g_x$ and $g_z$. This will be implemented for you using a method called ADAPT-VQE with pre-determined parameters. 
3. **Evolve under the Ising Hamiltonian.** With a large enough initial energy $E_{\text{tot}}>m + m$ (setting the speed of light $c=1$), an *inelastic collision* can occur in which a heavier, slower-moving particle with mass $M$ is produced. ***A goal of simulations like this is to observe this process.***
4. **Implement error mitigation and post-processing (PART II only).** These experiments operate at the *utility-scale*, requiring a large number of qubits and a substantial two-qubit gate depth. Error mitigation and post-processing are necessary to refine the experimental signal. 

</div> 

<div class="alert alert-block alert-warning">

### **Grading Criteria:** 
The bulk of the challenge will be graded based on values for the **vacuum-subtracted energy density** $E_n$. This quantity will be explained in detail later in this notebook.

#### **20 Points: $E_n$ for MPS simulation of $L = 15$**
- 5 points for correct wavepacket circuit construction (*PART I, Step 2c*)
- 5 points for $E_n$ results at $t = 0.0$ (*PART I, Step 2c*)
- 5 points for correct evolution circuit construction to $t = 4.0$ (*PART I, Step 3c*)
- 5 points for $E_n$ results at $t = 4.0$ (*PART I, Step 3c*)
#### **60 Points: $E_n$ for QPU simulation of $L = 100$**
- 30 points for $E_n$ results at $t = 2.5$ 
    - 10 points for correct circuit construction (*PART II, Step 3c*)
    - 5 points for raw expectation values (*PART II, Step 3e*)
    - 5 points for $E_n$ based on raw expectation values (*PART II, Step 3e*)
    - up to 10 points for $E_n$ based on best performance (comparison of submitted "best performance" file to theoretical simulation) (*PART II, Step 4d*)
- 30 points for results at $t = 14.0$
    - 10 points for correct circuit construction (*PART II, Step 3c*)
    - 5 points for raw expectation values (*PART II, Step 3e*)
    - 5 points for $E_n$ based on raw expectation values (*PART II, Step 3e*)
    - 10 points for $E_n$ based on best performance (comparison of submitted "best performance" file to theoretical simulation) (*PART II, Step 4d*)
#### **20 Points: Post-processed $E_n$ results**
- 10 points for post-processed results at $t = 2.5$ 
    - 5 points for correct implementation of ODR (*PART II, Step 4a*)
    - 2.5 points for correct post-processing based on symmetry (*PART II, Step 4b*)
    - 2.5 points for correct rescaling $E_n$ based on energy conservation (*PART II, Step 4c*)
- 10 points for post-processed results at $t = 14.0$ 
    - 5 points for correct implementation of ODR (*PART II, Step 4a*)
    - 2.5 points for correct post-processing based on symmetry (*PART II, Step 4b*)
    - 2.5 points for correct rescaling $E_n$ based on energy conservation (*PART II, Step 4c*)

#### **Winning Team**
The winning team with the highest total score will have the opportunity to present their solution notebook to the other contestants. 

#### *Good luck and have fun!*

</div> 

### **BEFORE YOU BEGIN, please follow the instructions in INSTALL.md to properly set up your environment.**

Then, fill in your `team_name` in the cell below and then run to load the required grader functions and create a folder to store your results:

In [ ]:
import os

team_name = "<TEAM_NAME>"

from qc_grader.challenges.qdc_2025 import (
    submit_name,
    grade_lab3_ex1, 
    grade_lab3_ex2, 
    grade_lab3_ex3, 
    grade_lab3_ex4, 
    grade_lab3_ex5,
    grade_lab3_ex6,
    grade_lab3_ex7,
    grade_lab3_ex8,
    grade_lab3_ex9,
    grade_lab3_ex10,
    grade_lab3_ex11,
    grade_lab3_ex12,
    grade_lab3_ex13,
    grade_lab3_ex14
)

submit_name(team_name)

folder_name = "submitted-results"
os.makedirs(folder_name, exist_ok=True)

# **PART I: MPS SIMULATION** (20 points)

## **PART I, Step 0: Specify the parameters of the system.**

Let's define our parameters:

- `L` : The number of lattice sites. This is equal to the number of qubits in our experiment.
- `d`: The size (spatial extent) of the initial wavepackets. We will pick both wavepackets to have the same size. This number must be odd and $\geq$ 3.
- `kzero` : The momentum around which the momentum profiles of the wavepackets are centered. We will pick both to have the same magnitude of momentum.
- `var_k`: The variance in momentum.

In addition, as noted above, we will fix the values of $g_{x}$ and $g_{z}$ to be the values used in the paper.


In PART I, we recommend using very modest values for lattice size and wavepacke sizes. The suggested values are  `L = 15` and `d = 3` -- please use these values for your MPS submission for grading purposes, but feel free to try out others. Note that as `L` increases, the computational cost will increase. The modest suggested values will keep the computation manageable and provide qualitative results, but will be **insufficient to resolve the inelastic scattering event**.

To match the paper results, please use `kzero` $= 0.36 \pi$ and `var_k` $=0.13^2$ for your submission. Again, feel free to play around with these values in PART I if desired.

In PART II, we will scale up the lattice size and wavepacket size considerably to increase the fidelity of the simulation -- due to the uncertainty principle, a large value for `d` (the spatial spread of wavepacket) is necessary to have a wavepacket with well-defined momentum, and a large value for `L` is needed to accomodate the larger value of `d` and provide enough "room" to observe the scattering event without significant finite size effects. 

In [ ]:
from math import pi 

# begin code

L = 
d = 

# end code

assert d % 2 == 1, "d must be odd"
assert d >= 3, "d must be >=3"

kzero = 0.36 * pi
var_k = 0.13**2

### DO NOT CHANGE gx and gz ###
gx = 1.25
gz = 0.15

## **PART I, Step 1: Prepare an initial state that has two wavepackets with definite momentum $k_0$.**

In particular, we want to prepare one ''left'' wavepacket with momentum $k_0$, and one ''right'' wavepacket with momentum $-k_0$. We will break this step up into manageable pieces. 

### **Step 1a: Prepare a wavepacket with momentum $k_0$ and size $d$.**

We have prepared a function `prep_wavepacket` that generates the wavepacket preparation circuit. Run the cell below to generate and view an example. In particular, this function returns a circuit acting on `d` qubits that produces a truncated wavepacket with momentum `kzero` and momentum variance `var_k`. The function also returns a vector `state_coeff` comprised of the magnitude of the probability amplitudes associated with the quantum state. 

Use the values for `L`, `d`, `kzero`, and `var_k` suggested above (or other values if you prefer :)) to generate a wavepacket. Now, pause for a second and think -- what's the first thing you should _always_ confirm to sanity check a quantum state?

*Check that the wavepacket is normalized!*

Finally, draw this circuit to get a feel for what it looks like. You can experiment with the decompose method to see what gates go into preparing this circuit.

In [ ]:
from utils import prep_wavepacket
import numpy as np

wp_circ, state_coeffs = prep_wavepacket(kzero=kzero, L=L, d=d, var_k=var_k)

# begin code

# Check that the wavepacket state is normalized

# end code

wp_circ.decompose().draw('mpl', fold=-1)

### **Step 1b: Prepare the "left" and "right" wavepackets with momentum $k_0$ and $-k_0$ (respectively) in a single circuit of size `L`.**

The left wavepacket will be centered at `left_center` and the right wavepacket will be centered at `right_center`. To reduce finite-size effects, we place the wavepackets away from the boundaries of the chain. To reduce initial interactions between wavepackets, we recommend placing the wavepackets such that there are three empty sites between them. For the recommended values of `L` and `d`, this is accomplished by using 4 and 10 for `left_center` and `right_center` (respectively). 

Fill in the arguments to the `prep_wavepacket` functions that appear below to create two wavepackets with opposite momentum, with the left wavepacket moving to the right and the right wavepacket moving to the left. *Note that this implies that their momenta will differ by a sign!*

In [ ]:
from qiskit import QuantumCircuit

left_center = 4
right_center = 10

# begin code 

left_wp_circ, state_coeffs = prep_wavepacket(kzero=, L=, d=, var_k=)
right_wp_circ, state_coeffs = prep_wavepacket(kzero=, L=, d=, var_k=)

# end code 

initial_wp_circ = QuantumCircuit(L)
initial_wp_circ.compose(left_wp_circ, list(range(left_center - (d - 1)//2, left_center + (d + 1)//2)), inplace=True)
initial_wp_circ.compose(right_wp_circ, list(range(right_center - (d - 1)//2, right_center + (d + 1)//2)), inplace=True)


Step 1 has been completed! Let's visualize the circuit with two initial wavepackets.

In [ ]:
initial_wp_circ.decompose().draw('mpl', fold=-1)

## **PART I, Step 2: Minimize the energy of the initial state without changing the momentum.**
For this step, we have provided a function called `build_adapt_vqe_state` that implements an energy minimization circuit on the lattice with pre-optimized parameters. In order to verify that the ADAPT-VQE circuit is working as expected, we need to build a set of observables that capture the site-specific *energy density* of the system. This will be an important observable -- measuring the energy density across the system over time will allow us to observe scattering.

### **Step 2a: Write a function to build the energy density as a function of site number $n$.**
Recall that the Hamiltonian of this theory is given by: $$H_{\text{ising}} = -\sum_{n=0}^{L-2} Z_{n} Z_{n+1} - \sum_{n=0}^{L-1}\big( g_{x} X_n + g_{z} Z_n \big).$$

We define the **energy density** of site $n$ in the "bulk" (that is, away from the boundary) as the expected value of $H_n$, where $H_n$ includes the single-site operators in the Hamiltonian and a symmetrized version of the interaction: $$H_n = -\frac{1}{2}\big( Z_{n-1} Z_{n} + Z_{n} Z_{n+1}\big) - g_x X_n - g_z Z_n .$$
For the boundary sites ($n = 0, L-1$), only one $ZZ$ interaction is included in $H_n$ and the factor of $1/2$ is dropped (open boundary conditions).

Fill in the function `ham_density` that generates $H_n$ (in the form of a `SparsePauliOp`) given the site index $n$. 

In [ ]:
from qiskit.quantum_info import Pauli, PauliList, SparsePauliOp

def ham_density(n:int, L:int, gx:float=gx, gz:float=gz)-> SparsePauliOp:
    assert n >=0
    assert n <= L-1
    id = Pauli('I'*L)
    
    # begin code
    
    return 
    # end code
    

# Check your work by viewing a few of the sparse Pauli operators: 
print(ham_density(0, L))
print(ham_density(1, L))
print(ham_density(L-1, L))

### **Step 2b: Generate the energy minimization circuit -- *Completed for you***

As prepared up to this point, the wavepackets include contributions from the higher particle number sectors of the Hilbert space. To project down to the single particle subspace, ADAPT-VQE is implemented to minimize the energy. Here, that entire step is performed by the `build_adapt_vqe_state` function. The details of this step are omitted but may be found in the paper linked above. The key to this step of wavepacket preparation is that the circuit ansatz does not alter the initial momentum prepared in Step 1. 

This step is already completed for you below.

In [ ]:
from utils import build_adapt_vqe_state

adapt_vqe_circ = build_adapt_vqe_state(L=L)

### **Step 2c: Verify that the energy minimization circuit lowers the energy of the prepared wavepackets.**

Let's compare the energy profile of the wavepackets before and after the ADAPT-VQE step. You should find that the energy profile after the ADAPT-VQE step is lower everywhere compared to the energy profile before the ADAPT-VQE step.

We will also prepare a vacuum state by applying the energy-minimizing ADAPT-VQE circuit to the all zero state and computing the resulting vacuum energy.

To run these simulations, we instantiate `AerSimulator` as the backend, as shown below. In the following line, instantiate the EstimatorV2 primitive from `qiskit-aer` with the `matrix_product_state` method by setting `"backend_options": {"method": "matrix_product_state"}` in the `options` field. 

In [ ]:
from qiskit_aer.primitives import EstimatorV2
from qiskit_aer import AerSimulator

aer_backend = AerSimulator(method='matrix_product_state')

# begin code

estimator_mps = #

# end code

Run the cell below to visualize the energy density before and after the application of ADAPT-VQE.

In [ ]:
import matplotlib.pyplot as plt
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

sim_pm = generate_preset_pass_manager(backend=aer_backend, optimization_level=3, seed_transpiler=111)

pre_avqe_wp_circ = initial_wp_circ.copy()

post_avqe_wp_circ = initial_wp_circ.copy()
post_avqe_wp_circ.compose(adapt_vqe_circ, qubits=range(L), inplace=True)

initial_vac_circ = QuantumCircuit(L)
post_avqe_vac_circ = initial_vac_circ.copy()
post_avqe_vac_circ.compose(adapt_vqe_circ, qubits=range(L), inplace=True)

pre_avqe_wp_circ = sim_pm.run(pre_avqe_wp_circ)
post_avqe_wp_circ = sim_pm.run(post_avqe_wp_circ)
post_avqe_vac_circ = sim_pm.run(post_avqe_vac_circ)

post_vqe_energies=[]
pre_vqe_energies=[]
vacuum_energies=[]
for site in range(L):
    obs = ham_density(site, L)
    job = estimator_mps.run([(post_avqe_wp_circ, obs),(pre_avqe_wp_circ, obs),(post_avqe_vac_circ, obs)])
    result=job.result()
    post_vqe_energies.append(result[0].data.evs)
    pre_vqe_energies.append(result[1].data.evs)
    vacuum_energies.append(result[2].data.evs)

plt.figure(figsize=(5, 4))
plt.plot(list(range(L)),pre_vqe_energies, label=f'Wavepackets Before ADAPT-VQE')
plt.plot(list(range(L)),post_vqe_energies, label=f'Wavepackets After ADAPT-VQE')
plt.plot(list(range(L)),vacuum_energies, label=f'Vacuum After ADAPT-VQE')
plt.xlabel('Lattice Site ' + r"$n$")
plt.ylabel(r"$E_{n}$")
plt.title('Energy Density')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left'); 

For the remainder of this challenge, we will focus on the **vacuum-subtracted energy density**, which is the difference between the energy density of the evolved wavepackets and that of the evolved vacuum. This shifted quantity is visualized below.

In [ ]:
mps_energies_t0 = np.array(post_vqe_energies) - np.array(vacuum_energies)

plt.figure(figsize=(5, 4))
plt.plot(list(range(L)), mps_energies_t0, label=f'Wavepackets After ADAPT-VQE')
plt.xlabel('Lattice Site ' + r"$n$")
plt.ylabel(r"$E_{n}$")
plt.title('Vacuum-Subtracted Energy Density')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left');

<div class="alert alert-block alert-info">

### **Save your MPS results (for `L=15`, `d=5`) for the wave packet preparation circuit (5 pts) and for the vacuum-subtracted energy density (5 pts) using the cell below, and then check your score.**

</div>

In [ ]:
from qiskit import qpy
np.save('submitted-results/mps_En_t0.npy', mps_energies_t0)

mps_wp_circ_t0 = post_avqe_wp_circ

with open("submitted-results/mps_wp_t0.qpy", "wb") as file:
    qpy.dump(mps_wp_circ_t0, file)

In [ ]:
grade_lab3_ex1(mps_energies_t0, mps_wp_circ_t0)

## **PART I, Step 3: Evolve under the Ising Hamiltonian.**
We will work in the Schrodinger picture, evolving both the wavepackets and the vacuum state in time to allow us to calculate the vacuum-subtracted energy density. (As an added bonus, the evolved vacuum state also serves as a **mitigation circuit** that will be used for error mitigation in PART II.)

To execute the time evolution, we need to create a Trotterized unitary that approximately time evolves under the Hamiltonian $H$: $U_{\text{Trotter}}(t) \approx e^{-iHt}$. Recall that the Ising model is governed by: $$H_{\text{ising}} = -\sum_{n=0}^{L-2} Z_{n} Z_{n+1} - \sum_{n=0}^{L-1}\big( g_{x} X_n + g_{z} Z_n \big).$$

To balance Trotterization error and 2-qubit gate depth, we will use a second-order Trotterization. In particular, we will use the Suzuki-Trotter formula: $$e^{-i\delta t(A + B)} \approx e^{-i\delta tB/2} e^{-i\delta tA} e^{-i\delta tB/2}.$$ 

We can minimize the 2-qubit gate depth by separately grouping the $Z$ and $X$ operators that appear in the Hamiltonian into quantites $A$ and $B$, respectively: $$A = -\sum_{n=0}^{L-2} Z_{n} Z_{n+1} - \sum_{n=0}^{L-1} g_{z} Z_n  \quad, \qquad B = -\sum_{n=0}^{L-1} g_{x} X_n.$$

Let's further split up $A$ into groups of operators that don't have any overlapping support, so they can be easily translated into low-depth quantum circuits: $$A_{\text{even}} = -\sum_{k = 0}^{\left \lfloor(L-2)/2\right \rfloor} Z_{2k} Z_{2k+1} \quad, \qquad A_{\text{odd}} = -\sum_{k = 1}^{\left \lfloor(L-1)/2\right \rfloor} Z_{2k-1} Z_{2k} \quad, \qquad A_{\text{local}} = -\sum_{n}^{L-1} g_z Z_{n} $$
$$A = A_{\text{even}} + A_{\text{odd}} + A_{\text{local}}.$$

These operators commute with each other and can therefore be split into separate unitaries with no additional errors: $$e^{A} = e^{A_{\text{even}}} e^{A_{\text{odd}}} e^{A_{\text{local}}}.$$

In total, the unitary for one second-order Trotter step $\delta t$ can be written as: $$U_2(\delta t) = e^{-i\delta tB/2} e^{-i\delta tA_{\text{even}}} e^{-i\delta tA_{\text{odd}}} e^{-i\delta tA_{\text{local}}} e^{-i\delta tB/2}.$$

### **Step 3a: Generate a circuit to evolve the system under one Trotter step.** 
Fill in the function `trotter_step_qc` below to generate a `QuantumCircuit` that implements evolution under one Trotter step $U_2(\delta t)$.

To implement $ZZ$ interactions with $\text{CZ}$ gates, you may find the following circuit identity helpful:
<div>
<img src="figs/circuit_identity.png" width="500"/>
</div>

Note that you will actually need to implement the exponentiated version of this equivalence, that is, one that relates $\text{Rzz}$ gates two $\text{CZ}$ gates.

<div class="alert alert-block alert-warning"> 

#### **Optional:** 
Consider a second implementation of the $ZZ$ interactions encoded in $A_{\text{even/odd}}$ on hardware: one that uses fractional $\text{Rzz}$ gates (https://quantum.cloud.ibm.com/docs/en/guides/fractional-gates) when `use_fractional_gates=True`. Using fractional gates decreases two-qubit gate depth, but complicates the application of some error mitigation techniques -- we will explore these subtleties in PART II.

</div>

In [ ]:
def trotter_step_qc(L:int, gx:float, gz:float, delta_t:float, use_fractional_gates:bool=False) -> QuantumCircuit: 
    # Return a circuit generating one Trotter step of time evolution.
    qc = QuantumCircuit(L)
    
    # begin code
    

        
    # end code
    
    return qc

### **Step 3b: Obtain the time-evolved states.**

Now that we can implement one Trotter step, we can generate a circuit that evolves under an arbitrary number of Trotter steps `num_trotter`. Fill in the function `obtain_time_evolved_state` that adds onto the state preparation circuit (given by the input `init_state`). The function should first minimize the energy via `adapt_vqe_circ`, then, the function should generate `num_trotter` steps of Trotter evolution with Trotter step size `delta_t`. The full `QuantumCircuit` object should be returned.

In [ ]:
def obtain_time_evolved_state(num_trotter:int, delta_t:float, L:int, gx: float=gx, gz: float=gz, init_state: QuantumCircuit=initial_wp_circ, adapt_vqe_circ: QuantumCircuit=adapt_vqe_circ, use_fractional_gates: bool=False) -> QuantumCircuit:
    # Starting with the initial circuit `init_state`, minimize the energy via `adapt_vqe_circ`, then evolve the circuit under `num_trotter` Trotter steps.
    # Return the full circuit. 
    evol_qc = init_state.copy()
    
    # begin code
    

        
    # end code
    
    return evol_qc

### **Step 3c: Verify that that particles move towards each other and scatter as they evolve in time.**
Run the cell below to evolve the wavepackets in time and observe the resulting energy density. You should observe the initial wavepackets as two peaks at $t=0$, which then move towards each other, collide (resulting in a single peak), and then move away from each other. Only two outgoing peaks will be visible at the resolution of this simulation, corresponding to the elastic scattering process. For `L=15`, this will take about 2 minutes to run.

In [ ]:
delta_t = 0.5
trotter_steps = [0, 2, 4, 6, 8]
energies_over_time = []
mps_wp_circs = []
mps_vac_circs = []

fig, ax = plt.subplots(nrows=len(trotter_steps), ncols=1, sharex='all', figsize=(5,1.5*len(trotter_steps)))
for idx, num_trotter in enumerate(trotter_steps):
    print(f'Number of Trotter steps: {num_trotter}')
    wp_circ = obtain_time_evolved_state(L=L, num_trotter=num_trotter, delta_t=delta_t, init_state=initial_wp_circ, adapt_vqe_circ=adapt_vqe_circ)
    mps_wp_circs.append(wp_circ)
    vac_circ = obtain_time_evolved_state(L=L, num_trotter=num_trotter, delta_t=delta_t, init_state=initial_vac_circ, adapt_vqe_circ=adapt_vqe_circ)
    mps_vac_circs.append(vac_circ)
    isa_wp_circ = sim_pm.run(wp_circ)
    isa_vac_circ = sim_pm.run(vac_circ)
    print(f'2q depth of circuit: {isa_wp_circ.depth(lambda x:len(x.qubits)==2)}')
    energies=[]
    for site in range(L):
        obs = ham_density(site, L)
        wp_obs = obs.apply_layout(isa_wp_circ.layout)
        vac_obs = obs.apply_layout(isa_vac_circ.layout)
        job = estimator_mps.run([(isa_wp_circ,wp_obs), (isa_vac_circ,vac_obs)])
        result=job.result()
        energies.append(result[0].data.evs - result[1].data.evs)
    energies_over_time.append(energies)
    
    ax[-1-idx].plot(list(range(L)),energies)
    ax[-1-idx].set_ylabel(r"$E_{n}$")
    ax[-1-idx].text(0.05, 0.95, f"t: {num_trotter*delta_t}", transform=ax[-1-idx].transAxes,
        verticalalignment='top')#, bbox=props)

ax[-1].set_xlabel('Lattice Site ' + r"$n$")
ax[0].set_title('Vacuum-Subtracted Energy Density')
plt.tight_layout(pad=0.5)

Run the cell below for an alternative visualization of the scattering process.

In [ ]:
import pandas as pd
import seaborn as sns

df = pd.DataFrame(energies_over_time[::-1], index=trotter_steps[::-1])

ax = sns.heatmap(df,cmap="viridis")
cbar = ax.collections[0].colorbar
cbar.set_label(r"$E_{n}$", fontsize=12)
plt.xlabel('Lattice Site ' + r"$n$", fontsize=10)
plt.ylabel("Trotter Step", fontsize=10)
plt.title("Energy Density over Time", fontsize=14)
plt.show()


<div class="alert alert-block alert-info">

### **Save your MPS results (for `L=15`, `d=5`) at `t=4.0` for the evolution circuits (2.5 pts each) and for the vacuum-subtracted energy density (5 pts) using the cell below, and then check your score.**

</div>

In [ ]:
mps_wp_circ_t4 = mps_wp_circs[-1]
mps_vac_circ_t4 = mps_vac_circs[-1]
mps_energies_t4 = np.array(energies_over_time[-1])

with open("submitted-results/mps_wp_t4.qpy", "wb") as file:
    qpy.dump(mps_wp_circ_t4, file)
with open("submitted-results/mps_vac_t4.qpy", "wb") as file:
    qpy.dump(mps_vac_circ_t4, file)

np.save('submitted-results/mps_En_t4.npy', mps_energies_t4)

In [ ]:
grade_lab3_ex2(mps_energies_t4, mps_wp_circ_t4, mps_vac_circ_t4)

# **PART II: HARDWARE**

Let's now prepare to run this on hardware. We will quickly repeat steps 0-3 with updated parameters, and then in step 4, we will implement some error mitigation. In order to use built-in error mitigation (in particular Pauli twirling), we will **not** use fractional gates. The use of fractional gates (along with the by-hand implementation of Pauli twirling with a reduced set of Pauli's) can be explored later as a "stretch goal."

<div class="alert alert-block alert-success">

### **You should go through Part II a total of two times to get results for two different evolution times `t`:**
- `t = 2.5` (5 Trotter steps) -- for this simulation, we provide MPS simulation results that you can compare your results to in case you need to de-bug anything. 
- `t = 14.0` (28 Trotter steps) -- for this simulation, we will NOT provide simulation results. 

</div>

## **PART II, Step 0: Specify the parameters of the system.**

We will now use `L = 100` with wavepackets of size `d = 21` to match the original paper. 

In [ ]:
L = 100
d = 21
assert d % 2 == 1, "d must be odd"
assert d >= 3, "d must be >=3"
kzero = 0.36 * pi
var_k = 0.13**2

## **PART II, Step 1: Prepare an initial state that has two wavepackets with definite momentum $k_0$.**

Fill in the arguments to the `prep_wavepacket` functions that appear below to create two wavepackets with opposite momentum. This takes a couple minutes to run!

In [ ]:
midpoint = (L-1)//2

left_center = midpoint - (d//2) - 2
right_center = midpoint + (d//2) + 2

# begin code

left_wp_circ, state_coeffs = prep_wavepacket(kzero=, L=, d=, var_k=)
right_wp_circ, state_coeffs = prep_wavepacket(kzero=, L=, d=, var_k=)

# end code

initial_wp_circ = QuantumCircuit(L)
initial_wp_circ.compose(left_wp_circ.to_gate(), list(range(left_center - (d - 1)//2, left_center + (d + 1)//2)), inplace=True)
initial_wp_circ.compose(right_wp_circ.to_gate(), list(range(right_center - (d - 1)//2, right_center + (d + 1)//2)), inplace=True)


For effective error mitigation, the vacuum circuit should have a structure (and therefore noise profile) that matches the wavepacket circuit as closely as possible. To achieve this, we will also insert some "wavepacket" circuits on the vacuum circuit -- but this time, we use the `prep_zero=True` keyword. This will generate wavepacket circuits with adjusted parameters that "do nothing" to the initial all zero state. 

Fill in the other arguments below -- this takes a couple minutes to run!

In [ ]:
# begin code 

left_vac_circ, state_coeffs = prep_wavepacket(kzero=, L=, d=, var_k=, prep_zero=True)
right_vac_circ, state_coeffs = prep_wavepacket(kzero=, L=, d=, var_k=, prep_zero=True)

# end code

initial_vac_circ = QuantumCircuit(L)
initial_vac_circ.compose(left_vac_circ.to_gate(), list(range(left_center - (d - 1)//2, left_center + (d + 1)//2)), inplace=True)
initial_vac_circ.compose(right_vac_circ.to_gate(), list(range(right_center - (d - 1)//2, right_center + (d + 1)//2)), inplace=True)


Visualize the wavepacket circuit below.

In [ ]:
initial_wp_circ.decompose().draw('mpl')

## **PART II, Step 2: Minimize the energy of the initial state without changing the momentum -- *Completed for you***

### **Step 2a: Generate Observables to be Measured -- *Completed for you***

We will implement an error mitigation technique called *Operator Decoherence Renormalization* (ODR), as was done by the original authors. To do so, we will need access to the individual single-site `X` and `Z` Pauli observables as well as the weight-2 `ZZ` Pauli observables before building quantities like the energy density and total energy. The already completed function below generates these Pauli observables and places them in a list.

In [ ]:
def generate_observables(L:int): 
    obs = []
    id = Pauli('I'*L)
    for n in range(L):
        term = id.copy()
        term[n] = Pauli('X')
        obs.append(term)
    
    for n in range(L):
        term = id.copy()
        term[n] = Pauli('Z')
        obs.append(term)
        
    for n in range(L-1): 
        term = id.copy()
        term[n] = Pauli('Z')
        term[n+1] = Pauli('Z')
        obs.append(term)
    
    return obs

obs = generate_observables(L)

### **Step 2b: Write functions to compute energy density and total energy from measured observables -- *Completed for you***

Let's prepare some functions that group the measured expectation values into our quantities of interest: energy density and total energy (the expectation value of the Hamiltonian). The functions below are already filled. They take as input an array of expected values (assuming the same ordering established in `generate_observables`) and return an array of energy densities (`compute_energy_density`) and the total energy (`compute_total_energy`). Recall the definitions: $$H_{\text{ising}} = -\sum_{n=0}^{L-2} Z_{n} Z_{n+1} - \sum_{n=0}^{L-1}\big( g_{x} X_n + g_{z} Z_n \big) \quad , \qquad H_n = -\frac{1}{2}\big( Z_{n-1} Z_{n} + Z_{n} Z_{n+1}\big) - g_x X_n - g_z Z_n ,$$
and for the boundary sites ($n = 0, L-1$), only one $ZZ$ interaction is included in $H_n$ and the factor of $1/2$ is dropped (open boundary conditions).

In [ ]:
from numpy.typing import NDArray
def compute_energy_density(result_evs:NDArray, L:int, gx:float=gx, gz:float=gz): 
    # Return a list of the expectation value of Hn for n in range(L). 
    # See `generate_observables` for the expected ordering of measured Pauli observables in result_evs.
    energy_density = []
    
    for n in range(L): 
        energy = - gx * result_evs[n] - gz * result_evs[L + n]
        if n == L-1: 
            energy += -1.0 * result_evs[2*L + n-1]
        elif n == 0: 
            energy += -1.0 * result_evs[2*L + n]
        else: 
            energy += -0.5 * result_evs[2*L + n-1] - 0.5 * result_evs[2*L + n]
        energy_density.append(energy)
    
    return np.array(energy_density)

def compute_total_energy(result_evs:NDArray, L:int, gx:float=gx, gz:float=gz): 
    # Return the expectation value of the Hamiltonian.
    # See `generate_observables` for the expected ordering of measured Pauli observables in result_evs.
    total_energy = 0.0
    
    for n in range(L): 
        total_energy += - gx * result_evs[n] - gz * result_evs[L + n]
        if n < L-1: 
            total_energy += -1.0 * result_evs[2*L + n]
            
    return total_energy

### **Step 2c: Generate the energy minimization circuit -- *Completed for you***

As in PART I, this step is already completed for you below.

In [ ]:
adapt_vqe_circ = build_adapt_vqe_state(L=L)

## **PART II, Step 3: Evolve under the Ising Hamiltonian -- *on Hardware!*** (up to 30 points per `t`)

In this step, we will prepare the physical circuits that are going to be run on hardware. Let's take our time to set this up correctly -- first, let's select a backend. As mentioned previously, for now, we will avoid using fractional gates.

### **Step 3a: Select Backend**

Fill in the name of your preferred backend below.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

use_fractional_gates = False
service = QiskitRuntimeService(name='qdc-2025')
backend = service.backend('<preferred-backend>', use_fractional_gates=use_fractional_gates)

### **Step 3b: Select `layout`**

Now, let's choose a layout. You may use the function `get_longest_path` to generate a long 1D chain of length `L` (takes a couple minutes to run), or you can build a chain by hand and take the current calibration data available online into account. One layout is provided for you below; it was created based on calibration data for `ibm_kingston` that may not be up to date! See the image below and note that the selected layout avoids edges with high two-qubit gate error. 

<div>
<img src="figs/layout.png" width="400"/>
</div>

<div class="alert alert-block alert-warning"> 

#### **Optional:** 
Update the hand-selected layout according to the current calibration data available for your selected QPU. 

</div>


In [ ]:
from utils import get_longest_path
# layout = get_longest_path(tuple(backend.coupling_map), num_qubits=L)

layout = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 
          19, 35, 
          34, 33, 
          39, 53, 
          54, 55, 
          59, 75, 
          74, 73, 
          79, 93, 
          92, 91, 90, 89, 
          78, 69, 
          70, 71, 
          58, 51, 
          50, 49, 
          38, 29, 
          28, 27, 26, 25, 24, 23, 22, 21, 
          36, 41, 
          42, 43, 44, 45, 46, 47, 
          57, 67, 
          66, 65, 64, 63, 62, 61, 
          76, 81, 
          82, 83, 84, 85, 86, 87, 
          97, 107, 
          108, 109, 
          118, 129, 
          128, 127, 126, 125, 
          117, 105, 
          104, 103, 102, 101, 
          116, 121, 
          122, 123, 
          136, 143, 
          142, 141]

print(len(layout))

### **Step 3c: Prepare `pubs`**

Next, we need to place the hardware circuits / observables into a list of `pubs` that will be run on the hardware. We have MPS simulation values for $t = 2.5$. Using a Trotter step $\delta t = 0.5$, this corresponds to `num_trotter = 5`. We recommend initially evolving to $t = 2.5$ (5 Trotter steps; *before* the collision). Later in this notebook, you will try $t = 14.0$ (28 Trotter steps; *after* the collision, but before evidence of an inelastic collision can be observed). 

As a quick check, for `CZ` gates, we expect a two-qubit gate depth of 65 for 5 Trotter steps and 157 for 28 Trotter steps. 

In [ ]:
delta_t = 0.5

# begin code 

# fill num_trotter in below 
num_trotter = 

# end code 

t = delta_t * num_trotter 

pm = generate_preset_pass_manager(backend=backend, optimization_level=3, seed_transpiler=111, initial_layout=layout, routing_method=None)

print(f'Number of Trotter steps: {num_trotter}')
wp_circ = obtain_time_evolved_state(L=L, num_trotter=num_trotter, delta_t=delta_t, init_state=initial_wp_circ, adapt_vqe_circ=adapt_vqe_circ, use_fractional_gates=use_fractional_gates)
vac_circ = obtain_time_evolved_state(L=L, num_trotter=num_trotter, delta_t=delta_t, init_state=initial_vac_circ, adapt_vqe_circ=adapt_vqe_circ, use_fractional_gates=use_fractional_gates)
isa_wp_circ = pm.run(wp_circ)
isa_vac_circ = pm.run(vac_circ)
print(f'2q depth of wp_circ: {isa_wp_circ.depth(lambda x:len(x.qubits)==2)}')
print(isa_wp_circ.count_ops())
print(f'2q depth of vac_circ: {isa_vac_circ.depth(lambda x:len(x.qubits)==2)}')
print(isa_vac_circ.count_ops())
isa_obs = [ob.apply_layout(isa_wp_circ.layout) for ob in obs]
pubs = [(isa_wp_circ, isa_obs), (isa_vac_circ, isa_obs)]


<div class="alert alert-block alert-info">

### **Save your hardware circuits at `t = 2.5` *before transpilation* for grading using the cell below (10 points for each `t`), and then check your score.**

</div>

In [ ]:
hardware_wp_circ = wp_circ.copy()
hardware_vac_circ = vac_circ.copy()

with open('submitted-results/hardware_wp_t' + str(np.abs(np.round(t, decimals=2))) + '.qpy', "wb") as file:
    qpy.dump(hardware_wp_circ, file)
with open('submitted-results/hardware_vac_t' + str(np.abs(np.round(t, decimals=2))) + '.qpy', "wb") as file:
    qpy.dump(hardware_vac_circ, file)

In [ ]:
grade_lab3_ex3(t, hardware_wp_circ, hardware_vac_circ)

### **Step 3d: Prepare `estimator`**

Next, let's set up the options for our hardware run using the `Estimator`. Consider enabling dynamical decoupling to reduce error on idle qubits. Enable [gate and measurement twirling](https://quantum.cloud.ibm.com/docs/en/guides/configure-error-mitigation) and use values for `num_randomizations` and `shots_per_randomization` recommended below. The options set below are similar to those used in the original paper (for $t \leq 15$), with half the number of twirls. Note that we will follow the original paper and implement operator decoherence renormalization (ODR), which is a technique to mitigate both gate and measurement errors.

Using these settings, expect around 3m 40s of QPU usage per run (to get vacuum + wavepacket results for a single value of `t`). Be mindful that each team has a budget of **30 minutes** of QPU time for this challenge!

In [ ]:
from qiskit_ibm_runtime import EstimatorV2
estimator = EstimatorV2(mode=backend)

# begin code 

# To reduce error on idle qubits, consider enabling dynamical decoupling 


# For error suppression, enable gate and measurement twirling


# end code

# To keep the QPU usage reasonable when setting the options below!
estimator.options.twirling.num_randomizations = 20 * 16
estimator.options.twirling.shots_per_randomization = 500

# Maximum qpu time (in seconds) -- DO NOT CHANGE
estimator.options.max_execution_time = 600

options = estimator.options

# A handy way to keep track of these settings
options


### **Step 3e: Run on Hardware** (15 points for each `t`)

We're ready to submit the jobs! Submit the `pubs` built above as jobs using the estimator.

In [ ]:
# submit pubs

# begin code

job = #

# end code

NOTE: *If you previously submitted a job and want to retrieve the results again, you can do so using the associated job id using `job = service.job("<job id>")`. If you do this, be sure to also set `t` to the time that the circuits associated with the job_id evolved to (`num_trotter * delta_t`) in order for the rest of the notebook to run without issue.*

Check the job status below:

In [ ]:
job.status()

Once the job is 'DONE', retrieve the results from the job. Place the wavepacket results in `wp_meas_results` and the vacuum results in `vac_meas_results`. From these results, fetch the expectation values of the measured observables and place them in `wp_meas_evs` and `vac_meas_evs`, respectively.

In [ ]:
# Retrieve results and expectation values

# begin code

meas_results = #
wp_meas_results = #
vac_meas_results = #
wp_meas_evs = #
vac_meas_evs = #

# end code

Let's take a look at the resulting energy density before we do any further error mitigation / post-processing:

In [ ]:
wp_energies = compute_energy_density(wp_meas_evs, L)
vac_energies = compute_energy_density(vac_meas_evs, L)
raw_energies = wp_energies - vac_energies

plt.figure(figsize=(5, 4))
plt.plot(list(range(L)), raw_energies, color='red', linestyle=':', label='Raw')
plt.grid()
plt.xlabel('Lattice Site ' + r"$n$")
plt.ylabel(r"$E_{n}$")
plt.title('Vacuum-Subtracted Energy Density')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left');

<div class="alert alert-block alert-info">

### **Save your raw expectation values (5 pts for each `t`) and vacuum-subtracted energy density (for `L=100`, `d=21`) at `t=2.5` based on the *raw expectation values* (5 pts for each `t`) using the cell below, and then check your score.**

</div>

In [ ]:
# 1. Raw expectation values

np.save('submitted-results/raw_wp_evs_t' + str(np.abs(np.round(t, decimals=2))) + '.npy', np.array(wp_meas_evs))
np.save('submitted-results/raw_vac_evs_t' + str(np.abs(np.round(t, decimals=2))) + '.npy', np.array(vac_meas_evs))

# 2. Raw energies

np.save('submitted-results/raw_En_t' + str(np.abs(np.round(t, decimals=2))) + '.npy', np.array(raw_energies))

In [ ]:
grade_lab3_ex4(t, wp_meas_evs, vac_meas_evs, raw_energies);

## **Step 4: Implement error mitigation and post-processing** (10 points for each `t`)
### **Step 4a: ODR** (5 points for each `t`)

Operator Decoherence Renormalization (ODR) involves computing a rescaling factor (or "signal strength") $p_{\hat{O}}$ for each (Pauli) observable $\hat{O}$. The values for $p_{\hat{O}}$ are determined using a "mitigation circuit"  whose noise profile is **as similar as possible** to the "physics circuit," but whose theoretical ("predicted") values for observables are known or can be computed precisely: $$p_{\hat{O}} = \frac{\langle \hat{O} \rangle_{\text{meas}}}{\langle \hat{O} \rangle_{\text{pred}}}.$$


The evolved vacuum circuit is a good candidate for the mitigation circuit because the vacuum state is an eigenstate of the Hamiltonian, so as it evolves, its properties (theoretically) remain unchanged; any change that is observed can be attributed to decoherence. Moreoever, these properties are easy to compute in simulation because the vacuum can be prepared with a shallow circuit, requiring only the constant-depth `adapt_vqe_circ`. The signal strengths for evolution up to time $t$ can then be computed as: $$p_{\hat{O}} = \frac{\langle \psi_{\text{vac}} | \hat{U}^{\dagger}_2(t) \hat{O} \hat{U}_2(t) |\psi_{\text{vac}}\rangle_{\text{meas}}}{\langle \psi_{\text{vac}} |\hat{O} |\psi_{\text{vac}} \rangle_{\text{pred}}}.$$


Let's generate the set of predicted observables by using the MPS simulator to prepare $|\psi_{\text{vac}}\rangle$. We will also go ahead and simulate the initial observables predicted for the set of wavepackets, so that we can compute the vacuum-subtracted total energy of the initial state. This will be important for some post-processing.

In [ ]:
sim_pm = generate_preset_pass_manager(backend=aer_backend, optimization_level=3, seed_transpiler=111)

vac_pred_circ = QuantumCircuit(L)
vac_pred_circ.compose(adapt_vqe_circ, qubits=range(L), inplace=True)
isa_vac_pred_circ = sim_pm.run(vac_pred_circ)
wp_pred_circ = initial_wp_circ.copy()
wp_pred_circ.compose(adapt_vqe_circ, qubits=range(L), inplace=True)
isa_wp_pred_circ = sim_pm.run(wp_pred_circ)
job = estimator_mps.run([(isa_wp_pred_circ,obs), (isa_vac_pred_circ,obs)])
pred_results = job.result()
wp_pred_results = pred_results[0]
vac_pred_results = pred_results[1]
wp_pred_evs = wp_pred_results.data.evs
vac_pred_evs = vac_pred_results.data.evs

We can view the predicted energy densities using the resulting expectation values and the function `compute_energy_density` -- see the cell below.

In [ ]:
wp_pred_energies = compute_energy_density(wp_pred_evs, L)
vac_pred_energies = compute_energy_density(vac_pred_evs, L)

plt.figure(figsize=(5, 4))
plt.plot(list(range(L)),vac_pred_energies, label='Predicted Vacuum')
plt.plot(list(range(L)),wp_pred_energies, label='Predicted Wavepackets')
plt.xlabel('Lattice site')
plt.ylabel(r"$E_{n}$")
plt.title('Energy profile of wavepackets before scattering.')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left');

Let's also compute the total (vacuum-subtracted) energy predicted by the MPS simulation:

In [ ]:
total_energy = compute_total_energy(wp_pred_evs, L) - compute_total_energy(vac_pred_evs, L)
print(total_energy)

We can also compute the (vacuum-subtracted) energy of the measured results -- note the difference from the predicted value above! We'll address this more in Step 4c.

In [ ]:
total_energy_meas = compute_total_energy(wp_meas_evs, L) - compute_total_energy(vac_meas_evs, L)
print(total_energy_meas)

Now, we have what we need to estimate signal strength $p_{\hat{O}}$ for each Pauli observable. Fill in the function `signal_strengths` to return an array of $p_{\hat{O}}$ given an array of predicted expectation values and an array of measured expectation values.

In [ ]:
def signal_strengths(pred_evs, meas_evs): 
    # Return an array of signal strengths given the predicted and measured expectation values.
    
    # begin code
    sig_strengths = #
    # end code
    
    return sig_strengths

Using the same relationship between predicted and measured observables, fill in the function below to estimate mitigated (predicted) observables given an array of signal strengths and measured observables.

In [ ]:
def ODR_obs(meas_evs, sig_strengths): 
    # Return an array of mitigated observables given the measured expectation values and signal strengths.
    
    # begin code
    mit_obs = #
    # end code
    
    return mit_obs

Finally, we can compute the ODR-mitigated observables for the "physics" circuit, using the signal strengths computed from "mitigation" circuit!

In [ ]:
sig_strengths = signal_strengths(vac_pred_evs, vac_meas_evs)

mit_wp_evs = ODR_obs(wp_meas_evs, sig_strengths)
mit_vac_evs = ODR_obs(vac_meas_evs, sig_strengths)

Let's visualize the raw and mitigated Pauli observables below. Note that for the vacuum, the mitigated observables will exactly equal the predicted values, because we used them to derive the signal strengths.

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=1, sharex='all', figsize=(5,7))
ax[0].plot(list(range(L)), wp_meas_evs[0:L], color='red', linestyle=':', label='Wave Packets')
ax[0].plot(list(range(L)), vac_meas_evs[0:L], color='blue', linestyle=':', label='Vacuum')
ax[0].plot(list(range(L)), mit_wp_evs[0:L], color='red', label='Mitigated Wave Packets')
ax[0].plot(list(range(L)), mit_vac_evs[0:L], color='blue', label='Mitigated Vacuum')
ax[1].plot(list(range(L)), wp_meas_evs[L:2*L], color='red', linestyle=':')
ax[1].plot(list(range(L)), vac_meas_evs[L:2*L], color='blue', linestyle=':')
ax[1].plot(list(range(L)), mit_wp_evs[L:2*L], color='red')
ax[1].plot(list(range(L)), mit_vac_evs[L:2*L], color='blue')
ax[2].plot(list(range(L-1)), wp_meas_evs[2*L:], color='red', linestyle=':')
ax[2].plot(list(range(L-1)), vac_meas_evs[2*L:], color='blue', linestyle=':')
ax[2].plot(list(range(L-1)), mit_wp_evs[2*L:], color='red')
ax[2].plot(list(range(L-1)), mit_vac_evs[2*L:], color='blue')

ax[2].set_xlabel(r"$n$")
ax[0].set_ylabel(r"$\langle X_n \rangle$")
ax[1].set_ylabel(r"$\langle Z_n \rangle$")
ax[2].set_ylabel(r"$\langle Z_n Z_{n+1} \rangle$")
ax[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left');

Next, let's take a look at the raw and mitigated values for the vacuum-subtracted energy density.

In the cell below, we provide some MPS simulation results for `t = 2.5` that we can compare to. 

In [ ]:
if np.abs(t) == 2.5:
    mps_energies = np.load('mps_energies_t2.5.npy')
else: 
    mps_energies = [np.nan for _ in range(L)]
    

In [ ]:
mit_wp_energies = compute_energy_density(mit_wp_evs, L)
mit_vac_energies = compute_energy_density(mit_vac_evs, L)
mit_energies = mit_wp_energies - mit_vac_energies

plt.figure(figsize=(5, 4))
plt.plot(list(range(L)), raw_energies, color='red', linestyle=':', label='Raw')
plt.plot(list(range(L)), mit_energies, color='red', label='Mitigated (ODR)')
plt.plot(list(range(L)), mps_energies, color='black', linestyle='--', label='MPS')
plt.grid()
plt.xlabel('Lattice Site ' + r"$n$")
plt.ylabel(r"$E_{n}$")
plt.title('Vacuum-Subtracted Energy Density')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left');

<div class="alert alert-block alert-info">

### **Save your vacuum-subtracted energy density (for `L=100`, `d=21`) at `t=2.5` based on the *ODR-mitigated expectation values* using the cell below (5 pts for each `t`), and then check your score.**

</div>

In [ ]:
# 3. ODR mitigated

np.save('submitted-results/mit_En_t' + str(np.abs(np.round(t, decimals=2))) + '.npy', np.array(mit_energies))

In [ ]:
grade_lab3_ex5(t, wp_meas_evs, vac_meas_evs, mit_energies);

### **Step 4b: Using Symmetry** (2.5 points for each `t`)

Hopefully, the mitigated signal resembles the MPS simulation -- but likely, it isn't perfect! There are a few other things we can do in post-processing to clean up the result. First, physically, we expect the vacuum-subtracted energy density to be mirrored about the collision point such that $E_n = E_{L - 2 - n}$. Fill in a function below that symmetrizes the vacuum-subtracted energy density by taking the mean of $E_n, E_{L - 2 - n}$. 

In [ ]:
def symmetrize_energy_density(energy_density): 
    # Return the symmeterized energy density.
    sym_energy_density = energy_density.copy()
    
    # begin code
    
    
    
    # end code
    
    return sym_energy_density

Let's take a look at the symmetrized results:

In [ ]:
mit_sym_energies = symmetrize_energy_density(mit_energies)

plt.figure(figsize=(5, 4))
plt.plot(list(range(L)), raw_energies, color='red', linestyle=':', label='Raw')
plt.plot(list(range(L)), mit_sym_energies, color='red', label='Sym. + Mitigated (ODR)')
plt.plot(list(range(L)), mps_energies, color='black', linestyle='--', label='MPS')
plt.grid()
plt.xlabel('Lattice Site ' + r"$n$")
plt.ylabel(r"$E_{n}$")
plt.title('Vacuum-Subtracted Energy Density')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left');

<div class="alert alert-block alert-info">

### **Save your *symmetrized* vacuum-subtracted energy density (for `L=100`, `d=21`) at `t=2.5` based on the *ODR-mitigated expectation values* using the cell below (2.5 pts for each `t`), and then check your score.**

</div>

In [ ]:
# 4. Mitigated + symmetrized

np.save('submitted-results/mit_sym_En_t' + str(np.abs(np.round(t, decimals=2))) + '.npy', np.array(mit_sym_energies))

In [ ]:
grade_lab3_ex6(t, wp_meas_evs, vac_meas_evs, mit_sym_energies);

### **Step 4c: Energy Rescaling** (2.5 points per `t`)

Lastly, we can rescale the energy density to correct for errors that change the total energy, which should remain constant through time evolution. In particular, we can rescale the vacuum-subtracted energy density so that the total energy matches that of the initial state: $$E_n = E_n^{(\text{ODR})} \frac{E_{\text{tot}}}{E_{\text{tot}}^{(\text{ODR})}}.$$

We already computed the total energy of the initial state. Below, let's compute the total energy of the ODR-mitigated expectation values and then rescale the energy density.

In [ ]:
# compute the rescaled energy density of the ODR-mitigated results. 

total_energy_mit = compute_total_energy(mit_wp_evs, L) - compute_total_energy(mit_vac_evs, L)

# begin code

rescaled_mit_sym_energies = #

# end code

Take a look at the plot below to see the energy rescaled results.

In [ ]:
plt.figure(figsize=(5, 4))
plt.plot(list(range(L)), raw_energies, color='red', linestyle=':', label='Raw')
plt.plot(list(range(L)), rescaled_mit_sym_energies, color='red', label='Rescaled + Sym. + Mitigated (ODR)')
plt.plot(list(range(L)), mps_energies, color='black', linestyle='--', label='MPS')
plt.grid()
plt.xlabel('Lattice Site ' + r"$n$")
plt.ylabel(r"$E_{n}$")
plt.title('Vacuum-Subtracted Energy Density')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left');

<div class="alert alert-block alert-info">

### **Save your *rescaled, symmetrized* vacuum-subtracted energy density (for `L=100`, `d=21`) at `t=2.5` based on the *ODR-mitigated expectation values* using the cell below (2.5 pts for each `t`), and then check your score.**

</div>

In [ ]:
# 5. Mitigated + symmetrized + rescaled

np.save('submitted-results/rescaled_mit_sym_En_t' + str(np.abs(np.round(t, decimals=2))) + '.npy', np.array(rescaled_mit_sym_energies))

In [ ]:
grade_lab3_ex7(t, wp_meas_evs, vac_meas_evs, rescaled_mit_sym_energies);

<div class="alert alert-block alert-success">

## **Congratulations on making it this far!** 🎉

</div>

<div class="alert alert-block alert-info">

### **Save your "best" energy density results at `t = 2.5` and the *transpiled circuits* used to produce them for grading using the cell below (up to 10 points for each `t`), and then check your score to see how closely your results match theory.**

This might be the same as the mitigated + symmetrized + rescaled results, or something else (see Stretch Goals at the end of the challenge notebook).

</div>

In [ ]:
# 6. Best -- 
# This defaults to the mitigated + symmetrized + rescaled results and the original hardware experiments they are based on, 
# but time-permitting you may decide to run additional experiments on hardware (see Stretch Goals). 
# If you do so and end up improving upon your original results, change the four lines below to save your "best" results!

best_energies = rescaled_mit_sym_energies
best_wp_circ = isa_wp_circ.copy()
best_vac_circ = isa_vac_circ.copy()
best_job = job

np.save('submitted-results/best_En_t' + str(np.abs(np.round(t, decimals=2))) + '.npy', np.array(best_energies))
with open('submitted-results/best_wp_t' + str(np.abs(np.round(t, decimals=2))) + '.qpy', "wb") as file:
    qpy.dump(best_wp_circ, file)
with open('submitted-results/best_vac_t' + str(np.abs(np.round(t, decimals=2))) + '.qpy', "wb") as file:
    qpy.dump(best_vac_circ, file)
import pickle
with open('submitted-results/job_inputs_t' + str(np.abs(np.round(t, decimals=2))) + '.pickle', "wb") as file:
    pickle.dump(best_job.inputs, file)


In [ ]:
grade_lab3_ex8(t, best_wp_circ, best_vac_circ, best_energies);

*Note -- if the grader is returning a score of zero, it may be due to outliers in your energy density data that are very far from the theoretical results. This might be caused by a particularly error-prone qubit or connection between qubits. Consult the current calibration data for your selected backend and try finding a better initial layout!*

# **PART II: HARDWARE -- *Repeating for `t=14.0`***

## **Step 1: Prepare an initial state that has two wavepackets with definite momentum $k_0$.**

There's no need to repeat this since `initial_wp_circ` and `initial_vac_circ` have already been generated.

## **PART II, Step 2: Minimize the energy of the initial state without changing the momentum -- *Completed for you***

There's no need to repeat this since `adapt_vqe_circ` has already been generated.

## **PART II, Step 3: Evolve under the Ising Hamiltonian -- *on Hardware!*** (up to 30 points per `t`)

In this step, we will prepare the physical circuits that are going to be run on hardware. Let's take our time to set this up correctly.

Below, prepare the hardware circuits / observables into a list of `pubs` that will be run on the hardware. For a Trotter step $\delta t = 0.5$, use `num_trotter = 28` in order to evolve to `t=14.0` (*after* the collision, but before evidence of an inelastic collision can be observed). 

As a quick check, for `CZ` gates, we expect a two-qubit gate depth of 157 for 28 Trotter steps. 

In [ ]:
delta_t = 0.5

# begin code

# fill num_trotter in below 
num_trotter = 

# end code

t = delta_t * num_trotter 

pm = generate_preset_pass_manager(backend=backend, optimization_level=3, seed_transpiler=111, initial_layout=layout, routing_method=None)

print(f'Number of Trotter steps: {num_trotter}')
wp_circ = obtain_time_evolved_state(L=L, num_trotter=num_trotter, delta_t=delta_t, init_state=initial_wp_circ, adapt_vqe_circ=adapt_vqe_circ, use_fractional_gates=use_fractional_gates)
vac_circ = obtain_time_evolved_state(L=L, num_trotter=num_trotter, delta_t=delta_t, init_state=initial_vac_circ, adapt_vqe_circ=adapt_vqe_circ, use_fractional_gates=use_fractional_gates)
isa_wp_circ = pm.run(wp_circ)
isa_vac_circ = pm.run(vac_circ)
print(f'2q depth of wp_circ: {isa_wp_circ.depth(lambda x:len(x.qubits)==2)}')
print(isa_wp_circ.count_ops())
print(f'2q depth of vac_circ: {isa_vac_circ.depth(lambda x:len(x.qubits)==2)}')
print(isa_vac_circ.count_ops())
isa_obs = [ob.apply_layout(isa_wp_circ.layout) for ob in obs]
pubs = [(isa_wp_circ, isa_obs), (isa_vac_circ, isa_obs)]

<div class="alert alert-block alert-info">

### **Save your hardware circuits at `t = 14.0` *before transpilation* for grading using the cell below (10 points for each `t`), and then check your score.**

</div>

In [ ]:
hardware_wp_circ = wp_circ.copy()
hardware_vac_circ = vac_circ.copy()

with open('submitted-results/hardware_wp_t' + str(np.abs(np.round(t, decimals=2))) + '.qpy', "wb") as file:
    qpy.dump(hardware_wp_circ, file)
with open('submitted-results/hardware_vac_t' + str(np.abs(np.round(t, decimals=2))) + '.qpy', "wb") as file:
    qpy.dump(hardware_vac_circ, file)

In [ ]:
grade_lab3_ex9(t, hardware_wp_circ, hardware_vac_circ);

We're ready to submit the jobs! Submit the `pubs` built above as jobs using the estimator.

In [ ]:
# submit pubs

# begin code

job = #

# end code

NOTE: *If you previously submitted a job and want to retrieve the results again, you can do so using the associated job id using `job = service.job("<job id>")`. If you do this, be sure to also set `t` to the time that the circuits associated with the job_id evolved to (`num_trotter * delta_t`) in order for the rest of the notebook to run without issue.*

Check the job status below:

In [ ]:
job.status()

Once the job is complete, retrieve the results from the job. Place the wavepacket results in `wp_meas_results` and the vacuum results in `vac_meas_results`. From these results, fetch the expectation values of the measured observables and place them in `wp_meas_evs` and `vac_meas_evs`, respectively.

In [ ]:
# Retrieve results and expectation values

# begin code

meas_results = #
wp_meas_results = #
vac_meas_results = #
wp_meas_evs = #
vac_meas_evs = #

# end code

Let's take a look at the resulting energy density:

In [ ]:
wp_energies = compute_energy_density(wp_meas_evs, L)
vac_energies = compute_energy_density(vac_meas_evs, L)
raw_energies = wp_energies - vac_energies

plt.figure(figsize=(5, 4))
plt.plot(list(range(L)), raw_energies, color='red', linestyle=':', label='Raw')
plt.grid()
plt.xlabel('Lattice Site ' + r"$n$")
plt.ylabel(r"$E_{n}$")
plt.title('Vacuum-Subtracted Energy Density')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left');

<div class="alert alert-block alert-info">

### **Save your raw expectation values (5 pts for each `t`) and vacuum-subtracted energy density (for `L=100`, `d=21`) at `t=14.0` based on the *raw expectation values* (5 pts for each `t`) using the cell below, and then check your score.**

</div>

In [ ]:
# 1. Raw expectation values

np.save('submitted-results/raw_wp_evs_t' + str(np.abs(np.round(t, decimals=2))) + '.npy', np.array(wp_meas_evs))
np.save('submitted-results/raw_vac_evs_t' + str(np.abs(np.round(t, decimals=2))) + '.npy', np.array(vac_meas_evs))

# 2. Raw energies

np.save('submitted-results/raw_En_t' + str(np.abs(np.round(t, decimals=2))) + '.npy', np.array(raw_energies))

In [ ]:
grade_lab3_ex10(t, wp_meas_evs, vac_meas_evs, raw_energies);

## **Step 4: Implement error mitigation and post-processing** (10 points for each `t`)
### **Step 4a: ODR** (5 points for each `t`)

In [ ]:
sig_strengths = signal_strengths(vac_pred_evs, vac_meas_evs)

mit_wp_evs = ODR_obs(wp_meas_evs, sig_strengths)
mit_vac_evs = ODR_obs(vac_meas_evs, sig_strengths)

Let's visualize the raw and mitigated Pauli observables below. Note that for the vacuum, the mitigated observables will exactly equal the predicted values, because we used them to derive the signal strengths.

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=1, sharex='all', figsize=(5,7))
ax[0].plot(list(range(L)), wp_meas_evs[0:L], color='red', linestyle=':', label='Wave Packets')
ax[0].plot(list(range(L)), vac_meas_evs[0:L], color='blue', linestyle=':', label='Vacuum')
ax[0].plot(list(range(L)), mit_wp_evs[0:L], color='red', label='Mitigated Wave Packets')
ax[0].plot(list(range(L)), mit_vac_evs[0:L], color='blue', label='Mitigated Vacuum')
ax[1].plot(list(range(L)), wp_meas_evs[L:2*L], color='red', linestyle=':')
ax[1].plot(list(range(L)), vac_meas_evs[L:2*L], color='blue', linestyle=':')
ax[1].plot(list(range(L)), mit_wp_evs[L:2*L], color='red')
ax[1].plot(list(range(L)), mit_vac_evs[L:2*L], color='blue')
ax[2].plot(list(range(L-1)), wp_meas_evs[2*L:], color='red', linestyle=':')
ax[2].plot(list(range(L-1)), vac_meas_evs[2*L:], color='blue', linestyle=':')
ax[2].plot(list(range(L-1)), mit_wp_evs[2*L:], color='red')
ax[2].plot(list(range(L-1)), mit_vac_evs[2*L:], color='blue')

ax[2].set_xlabel(r"$n$")
ax[0].set_ylabel(r"$\langle X_n \rangle$")
ax[1].set_ylabel(r"$\langle Z_n \rangle$")
ax[2].set_ylabel(r"$\langle Z_n Z_{n+1} \rangle$")
ax[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left');

Take a look at the raw and ODR-mitigated vacuum-subtracted energy density using the cell below.

In [ ]:
mit_wp_energies = compute_energy_density(mit_wp_evs, L)
mit_vac_energies = compute_energy_density(mit_vac_evs, L)
mit_energies = mit_wp_energies - mit_vac_energies

plt.figure(figsize=(5, 4))
plt.plot(list(range(L)), raw_energies, color='red', linestyle=':', label='Raw')
plt.plot(list(range(L)), mit_energies, color='red', label='Mitigated (ODR)')
plt.grid()
plt.xlabel('Lattice Site ' + r"$n$")
plt.ylabel(r"$E_{n}$")
plt.title('Vacuum-Subtracted Energy Density')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left');

<div class="alert alert-block alert-info">

### **Save your vacuum-subtracted energy density (for `L=100`, `d=21`) at `t=14.0` based on the *ODR-mitigated expectation values* using the cell below (5 pts for each `t`), and then check your score.**

</div>

In [ ]:
# 3. ODR mitigated

np.save('submitted-results/mit_En_t' + str(np.abs(np.round(t, decimals=2))) + '.npy', np.array(mit_energies))

In [ ]:
grade_lab3_ex11(t, wp_meas_evs, vac_meas_evs, mit_energies);

### **Step 4b: Using Symmetry** (2.5 points for each `t`)

In [ ]:
mit_sym_energies = symmetrize_energy_density(mit_energies)

plt.figure(figsize=(5, 4))
plt.plot(list(range(L)), raw_energies, color='red', linestyle=':', label='Raw')
plt.plot(list(range(L)), mit_sym_energies, color='red', label='Sym. + Mitigated (ODR)')
plt.grid()
plt.xlabel('Lattice Site ' + r"$n$")
plt.ylabel(r"$E_{n}$")
plt.title('Vacuum-Subtracted Energy Density')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left');

<div class="alert alert-block alert-info">

### **Save your *symmetrized* vacuum-subtracted energy density (for `L=100`, `d=21`) at `t=14.0` based on the *ODR-mitigated expectation values* using the cell below (2.5 pts for each `t`), and then check your score.**

</div>

In [ ]:
# 4. Mitigated + symmetrized

np.save('submitted-results/mit_sym_En_t' + str(np.abs(np.round(t, decimals=2))) + '.npy', np.array(mit_sym_energies))

In [ ]:
grade_lab3_ex12(t, wp_meas_evs, vac_meas_evs, mit_sym_energies);

### **Step 4c: Energy Rescaling** (2.5 points per `t`)

Lastly, we can do one more rescaling to correct for errors that change the total energy, which should remain constant through time evolution. In particular, we can rescale the vacuum-subtracted energy density so that the total energy matches that of the initial state: $$E_n = E_n^{(\text{ODR})} \frac{E_{\text{tot}}}{E_{\text{tot}}^{(\text{ODR})}}.$$

We already computed the total energy of the initial state. Below, let's compute the total energy of the ODR-mitigated expectation values and then rescale the energy density.

In [ ]:
# compute the rescaled energy density of the ODR-mitigated results. 

total_energy_mit = compute_total_energy(mit_wp_evs, L) - compute_total_energy(mit_vac_evs, L)

# begin code

rescaled_mit_sym_energies = #

# end code

View the energy rescaled results below:

In [ ]:
plt.figure(figsize=(5, 4))
plt.plot(list(range(L)), raw_energies, color='red', linestyle=':', label='Raw')
plt.plot(list(range(L)), rescaled_mit_sym_energies, color='red', label='Rescaled + Sym. + Mitigated (ODR)')
plt.grid()
plt.xlabel('Lattice Site ' + r"$n$")
plt.ylabel(r"$E_{n}$")
plt.title('Vacuum-Subtracted Energy Density')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left');

<div class="alert alert-block alert-info">

### **Save your *rescaled, symmetrized* vacuum-subtracted energy density (for `L=100`, `d=21`) at `t=14.0` based on the *ODR-mitigated expectation values* using the cell below (2.5 pts for each `t`), and then check your score.**

</div>

In [ ]:
# 5. Mitigated + symmetrized + rescaled

np.save('submitted-results/rescaled_mit_sym_En_t' + str(np.abs(np.round(t, decimals=2))) + '.npy', np.array(rescaled_mit_sym_energies))

In [ ]:
grade_lab3_ex13(t, wp_meas_evs, vac_meas_evs, rescaled_mit_sym_energies);

<div class="alert alert-block alert-info">

### **Save your "best" energy density results at `t = 14.0` and the *transpiled circuits* used to produce them for grading using the cell below (up to 10 points for each `t`), and then check your score to see how closely your results match theory.**

This might be the same as the mitigated + symmetrized + rescaled results, or something else (see Stretch Goals at the end of the challenge notebook).

</div>

In [ ]:
# 6. Best -- 
# This defaults to the mitigated + symmetrized + rescaled results and the original hardware experiments they are based on, 
# but time-permitting you may decide to run additional experiments on hardware (see Stretch Goals). 
# If you do so and end up improving upon your original results, change the four lines below to save your "best" results!

best_energies = rescaled_mit_sym_energies
best_wp_circ = isa_wp_circ.copy()
best_vac_circ = isa_vac_circ.copy()
best_job = job

np.save('submitted-results/best_En_t' + str(np.abs(np.round(t, decimals=2))) + '.npy', np.array(best_energies))
with open('submitted-results/best_wp_t' + str(np.abs(np.round(t, decimals=2))) + '.qpy', "wb") as file:
    qpy.dump(best_wp_circ, file)
with open('submitted-results/best_vac_t' + str(np.abs(np.round(t, decimals=2))) + '.qpy', "wb") as file:
    qpy.dump(best_vac_circ, file)
import pickle
with open('submitted-results/job_inputs_t' + str(np.abs(np.round(t, decimals=2))) + '.pickle', "wb") as file:
    pickle.dump(best_job.inputs, file)

In [ ]:
grade_lab3_ex14(t, best_wp_circ, best_energies);

*Note -- if the grader is returning a score of zero, it may be due to outliers in your energy density data that are very far from the theoretical results. This might be caused by a particularly error-prone qubit or connection between qubits. Consult the current calibration data for your selected backend and try finding a better initial layout!*

<div class="alert alert-block alert-warning"> 

## **Stretch Goals**
Hopefully, you have observed some signal of scattering. You may have observed that as $t$ is increased and the circuits become deeper, the measured signal becomes increasingly noisy. To see *inelastic scattering* requires going to $t \geq 25$ -- requiring a large circuit depth and sophisticated methods of error mitigation to combat the noise. 

For the remainder of the challenge, you have freedom to try and improve your results. One possible path that we have alluded to is the use of fractional gates ($R_{ZZ}$ rotations) in the Trotter steps, cutting the two-qubit gate depth of the time-evolution circuit in half. However, currently, using the `Estimator` to implement twirling with fractional gates is not supported -- yet twirling is essential for the use of ODR! It is possible to implement twirling "by hand" using a reduced set of Paulis; see, e.g, Fig. 15 of https://arxiv.org/pdf/2505.03111. Note also that due to the overall negative sign in the Hamiltonian, the required $R_{ZZ}$ rotations will have an argument $\theta$ outside of the allowed range for native implementation when evolving forward in time. Additional single-qubit gates will arise in transpilation to account for this. To avoid this additional single-qubit gate overhead, the original authors opted to evolve *backwards* in time ($\delta_t < 0$). This requires that the wavepackets be set up with swapped initial momentum (left wavepacket with negative momentum and right wavepacket with positive momentum) in order to observe scattering. 

Another pathway might be adjusting $\delta_t$ to explore the tradeoff between Trotterization error and decoherence due to large two-qubit gate depth, or increasing the number of twirls, or exploring other methods of error mitigation. 

</div>